# Lab 4: Visualizing Data
In this lab, you will learn how to use Python to visualize and explore data. Before creating analytical models, a data scientist must develop an understanding of the properties and relationships in a dataset. There are two goals for data exploration and visualization. First to understand the relationships between the data columns. Second to identify features that may be useful for predicting labels in machine learning projects. Additionally, redundant, collinear features can be identified. Thus, visualization for data exploration is an essential data science skill.
In this lab you will explore two datasets.

Your first goal is to explore a dataset that includes information about automobiles, which you want to use to create a solution that predicts the price of an automobile based on its characteristics. This type of predictive modeling, in which you attempt to predict a real numeric value, is known as regression; and it will be discussed in more detail later in the course â for now, the focus of this lab is on visually exploring the data to determine which features may be useful in predicting automobile prices.

After exploring the automobile data, you will turn your attention to some adult census data, which you plan to use to classify people as high or low income based information known about them. This technique of predicting whether data entities belong to one class or another is known as classification, and will be discussed later in the course. 

## Exercise 1: Visualizing Automobile Data for Regression
Python supports the matplotlib library; which provides extensive graphical capabilities.  Additionally, the Python Pandas library adds higher level graphics capability. These features make Python a useful language to create visualizations of your data when exploring relationships between the data features.  Further, you can identify features that may be useful for predicting labels in machine learning projects.

### Load the Automobiles Dataset
The automobiles dataset contains details of thousands of cars, some of which may be useful in predicting the car's price. Run the code in the following cell to load the data and view the data types of the columns in the DataFrame.

In [ ]:
%matplotlib inline

from azureml import Workspace
import pandas as pd
import numpy as np

ws = Workspace()
ds = ws.datasets['Automobile price data (Raw)']
auto_price = ds.to_dataframe()

## Convert some columns to numeric values
cols = ['price', 'bore', 'stroke', 'horsepower', 'peak-rpm']
auto_price[cols] = auto_price[cols].apply(pd.to_numeric, args=('coerce',))

## Remove rows with missing values
auto_price.dropna(axis = 0, inplace = True)

## Compute the log of the auto price
auto_price['lnprice'] = np.log(auto_price.price)

## Create a column with new levels for the number of cylinders
auto_price['num-cylinders'] = ['four-or-less' if x in ['two', 'three', 'four'] else 
    ('five-six' if x in ['five', 'six'] else 
    'eight-twelve') for x in auto_price['num-of-cylinders']]


auto_price.dtypes

Examine names and data types of the columns in this dataset. There are both numeric columns and character columns. In addition, you can see the first few values of each column. Character data is typically treated as categorical where each unique string value is a categorical value of the column. When exploring data sets, it is important to understand the relationships between each category and the label values.

### Create a Pair-Wise Scatter Plot
First, you will apply a visualization technique known as a scatter plot matrix to the auto.price dataset. Scatter plot matrix methods are widely used to quickly produce a single overall view of the relationships in a dataset.

The scatter plot matrix allows you to examine the relationships between many variables in one view. The alternative to viewing a scatter plot matrix is an examining a large number of scatter plot combinations one at a time. This process is both tedious, and in all likelihood difficult, since you need to remember relationships you have already viewed to understand the overall structure of the data.

Run the cell below to create a scatter plot matrix of the numeric features in the dataset.

In [ ]:
## Create pair-wise scatter plots         
def auto_pairs(plot_cols, df):
    import matplotlib.pyplot as plt
    from pandas.tools.plotting import scatter_matrix
    fig = plt.figure(figsize=(12, 12))
    fig.clf()
    ax = fig.gca()
    scatter_matrix(df[plot_cols], alpha=0.3, 
               diagonal='kde', ax = ax)
    return 'Done' 

## Numeric columns
plot_cols = ["wheel-base",
              "width",
              "height",
              "curb-weight",
              "engine-size",
              "bore",
              "compression-ratio",
              "city-mpg",
              "price",
              "lnprice"]

auto_pairs(plot_cols, auto_price)

Review the scatter ploy matrix (if the plot is too large for the cell, you can expand the cell by clicking its left margin).

Note that this plot is comprised of a number of scatter plots. For each variable there is both a row and a column. The variable is plotted on the vertical axis in the row, and on the horizontal axis in the column. In this way, every combination of cross plots for all variables is displayed in both possible orientations. Examine scatter plot matrix, which shows plots of each numeric column verses every other numeric column, and note the following:
- Not surprisingly the features, price and lnprice (log of price) are closely related. 
- Many features show significant collinearity, such as wheel.base, width and curb.weight, or curb.weight and engine.size, and city,mpg.
- A number of features show a strong relationship with the label, price, such as city.mpg, engine.size, and curb.weight.
- The feature, compression.ratio, has two distinct groupings of values, apparently corresponding to diesel (high compression ratio) and gasoline (low compression ratio) engines. These tightly grouped sets of values act very much like a categorical variable.
 
Note: The number of scatter plots and the memory required to compute and display them can be a bit daunting. You may wish to make a scatter plot matrix with fewer columns. For example, you can eliminate columns which are collinear with other columns such as highway.mpg, by using only city mpg. 

### Create Histograms
Histograms are a basic, yet powerful, tool for examining the distribution properties of a dataset. 

Note: You have already worked with histograms in a previous lab, so here you will focus on conditioned histograms of certain columns. A conditioned histogram is a histogram of a subset of data conditioned on another variable in the dataset. Often the histogram of a numeric variable is conditioned on a categorical variable. It is also possible to condition a histogram on (generally overlapping) ranges of a numeric variable. 

Run the cell below to define a function that creates histograms for a specified column, conditioned on the drive.wheel column; and then call the function for five features to see the results.

In [ ]:
## Function to plot conditioned histograms
def cond_hists(df, plot_cols, grid_col):
    import matplotlib.pyplot as plt
    import seaborn as sns
    ## Loop over the list of columns
    for col in plot_cols:
        grid1 = sns.FacetGrid(df, col=grid_col)
        grid1.map(plt.hist, col, alpha=.7)
    return grid_col

## Define columns for making a conditioned histogram
plot_cols2 = ["length",
               "curb-weight",
               "engine-size",
               "city-mpg",
               "price"]

cond_hists(auto_price, plot_cols2, 'drive-wheels')

Examine this series of conditioned plots, and note the following:
- There is a consistent difference in the distributions of the numeric features conditioned on the categories of drive.wheels. 
- The distribution of the values generally increases for length and curb.weight for real wheel drive (rwd) cars, with the values for 4 wheel drive (4wd) and real wheel drive (rwd) overlapping. 
- Cars with fwd have the highest city.mpg, with 4wd and rwd in a similar range. 
- Generally, 4wd cars have the lowest price, with rwd cars having the widest range. 

Note: In a previous lab you worked with histograms conditioned on number of cylinders. In this exercise you have created histograms conditioned on the type of drive wheels. In each case, the conditioning has highlighted different aspects of the relations in these data. Exploring the distributions in the dataset conditioned on other features will likely highlight other aspects of the structure of these data. You may wish to try this on your own. 

### Create Box Plots
Box plots are another tool for comparing distributions of conditioned numeric variables. Box plots allow comparison of summary statistics, median and quartiles, as well as to visualize outliers. 

Run the cell below to define a function that creates box plots conditioned on drive.wheels, and then call the function for the same columns as the histograms above.

In [ ]:
## Create boxplots of data
def auto_boxplot(df, plot_cols, by):
    import matplotlib.pyplot as plt
    for col in plot_cols:
        fig = plt.figure(figsize=(9, 6))
        ax = fig.gca()
        df.boxplot(column = col, by = by, ax = ax)
        ax.set_title('Box plots of ' + col + ' by ' + by)
        ax.set_ylabel(col)
    return by 

auto_boxplot(auto_price, plot_cols2, "drive-wheels")

Review each of the box plots. The conclusions you can draw from these plots are much the same as from the conditioned histograms. One detail now stands out; the curb.weight of 4wd cars is mostly between that of fwd and rwd cars.  As is often the case, a different view or visualization for the same data results in finding new relationships.

### Create Scatter Plots
Scatter plots are widely used to examine the relationship between two variables. In this procedure, you will explore methods to show the relationship between more than two variables on a two-dimensional scatter plot. First you will apply color as a method to examine multiple dimensions. By adding color to a scatter plot, you can effectively project three dimensions onto a two-dimensional plot. 

Run the cell below to define a function that creates scatter plots showing price vs a specified column, with color indicating the number of cylinders; and then call the function for four columns.

In [ ]:
## Create scatter plot
def auto_scatter(df, plot_cols):
    import matplotlib.pyplot as plt
    for col in plot_cols:
        fig = plt.figure(figsize=(8, 8))
        ax = fig.gca()
        temp1 = df.ix[df['fuel-type'] == 'gas']       
        temp2 = df.ix[df['fuel-type'] == 'diesel']
        if temp1.shape[0] > 0:                    
            temp1.plot(kind = 'scatter', x = col, y = 'price' , 
                           ax = ax, color = 'DarkBlue')                          
        if temp2.shape[0] > 0:                    
            temp2.plot(kind = 'scatter', x = col, y = 'price' , 
                           ax = ax, color = 'Red') 
        ax.set_title('Scatter plot of price vs. ' + col)
    return plot_cols

## Define columns for making scatter plots
plot_cols3 = ["length",
               "curb-weight",
               "engine-size",
               "city-mpg"]

auto_scatter(auto_price, plot_cols3)

Examine these plots, and note the following:
- Each of the four variables plotted against price shows a strong trend, indicating they might be good predictors of price.
- In each plot, the trend and positioning of the points for by numbers of cylinders are different. Cars with different numbers of cylinders seem to form distinct but overlapping clusters. 


### Create Conditioned Scatter Plots
Next you will create and examine conditioned scatter plots. By conditioning multiple dimensions, you can project several additional dimensions onto the two-dimensional plot.
You will not use point shape as a differentiator in this exercise, but keep in mind that shape can be as useful as color. Additionally, shape may be easier for the significant fraction of the population who are color blind. 

Note: Be careful when combining methods for projecting multiple dimensions. You can easily end up with a plot that is not only hard to interpret, but even harder for you to communicate your observations to your colleagues. For example, if you use three conditioning variables, plus color and shape, you are projecting seven dimensions of your dataset. While this approach might reveal important relationships, it may just create a complex plot. 

Run the cell below to define a function that creates scatter plots of price vs a specified column, conditioned by the number of cylinders and body style

In [ ]:
def condPltsCol(df, col1, col2, var1, var2):
    import matplotlib.pyplot as plt
    
    ## Find the levels of the conditioning variables
    levs1 = df[var1].unique().tolist()
    num1 = len(levs1)
    levs2 = df[var2].unique().tolist()
    num2 = len(levs2)   
    
    ## Determine the limits for the plots
    xlims = (df[col1].min(), df[col1].max())
    ylims = (df[col2].min(), df[col2].max())
    
    ## Define a figure and axes for the plot
    fig, ax = plt.subplots(num1, num2, figsize = (12, 8))

    ## Loop over conditioning variables subset the data
    ## and set create scatter plots for each conditioning
    ## variable pair and with data both gas and diesel cars    
    for i, val1 in enumerate(levs1):
        for j, val2 in enumerate(levs2):
            temp1 = df.ix[(df[var1] == val1) & (df[var2] == val2) & (df['fuel-type'] == 'gas')]       
            temp2 = df.ix[(df[var1] == val1) & (df[var2] == val2) & (df['fuel-type'] == 'diesel')]
            if temp1.shape[0] > 0:                    
                temp1.plot(kind = 'scatter', x = col1, y = col2 , ax = ax[i,j],
                          xlim = xlims, ylim = ylims, color = 'DarkBlue')                          
            if temp2.shape[0] > 0:                    
                temp2.plot(kind = 'scatter', x = col1, y = col2 , ax = ax[i,j],
                          xlim = xlims, ylim = ylims, color = 'Red')    
            ax[i,j].set_title(val1 + ' and ' + val2 )
            ax[i,j].set_xlabel('')
    
    ## Some lables for the x axis    
    ax[i,j].set_xlabel(col1)
    ax[i,(j-1)].set_xlabel(col1)
    return col1, col2

## Create conditioned scatter plots
def auto_scatter_cond(df, plot_cols, y, cond_var1, cond_var2):
    for col in plot_cols:
        condPltsCol(df, col, y, cond_var1, cond_var2)
    return cond_var1, cond_var2

auto_scatter_cond(auto_price, plot_cols3, 'price', 'body-style', 'num-cylinders')

Carefully examine the plots you have created. There is a lot of detail to understand. Generally, you should be able to conclude the following:
- There are no diesel cars for some of the conditioning combinations, such as convertables and cars with eight or more cylinders. 
- There are no cars at all for some combinations of conditioning variables.
- Fuel type does not seem to be a good predictor of price in most cases, since the points for diesel cars are mostly on the same trend as gasoline cars.
- Diesel engines are generally used in heavier cars, which then tend to be more fuel efficient. 
- As observed previously, the number of cylinders has a significant impact on price.
- There is considerable overlap in price conditioned by body style, indicating body style will be a weak predictor.  

By now, you should realize that exploring a dataset in detail is an open-ended and complex task. You may wish to try other combinations of conditioning variables, and color variable, to find some other interesting relationships in these data. 


## Exercise 2:  Visualizing Adult Census Data for Classification
In this exercise, you will explore some adult census data, which you plan to use for a classification solution in which you classify people has low income if they earn 50,000 or less, or high income if they earn more than 50,000. 

### Load the Adult Census Data
Run the cell below to load the adult census data.

In [ ]:
%matplotlib inline

from azureml import Workspace
import pandas as pd
import numpy as np

ws = Workspace()
ds = ws.datasets['Adult Census Income Binary Classification dataset']
Income = ds.to_dataframe()

Income.dtypes

Note the output which is a summary of the **Income** data frame. There are both numeric columns and character columns. In addition, you can see the first few values of each column.

Note: Character data is typically treated as categorical where each unique string value is a categorical value of the column. When exploring data sets, it is important to understand the relationships between each category and the label values.

### Bar Plot the Categorical Features
Run the cell below to define a function that creates a bar plot of non-numeric features vs income, and then use the function to plot the features in the Income dataset.

In [ ]:
## Plot categorical variables as bar plots
def income_barplot(df):
    import numpy as np
    import matplotlib.pyplot as plt
    
    cols = df.columns.tolist()[:-1]
    for col in cols:
        if(df.ix[:, col].dtype not in [np.int64, np.int32, np.float64]):
            temp1 = df.ix[df['income'] == '<=50K', col].value_counts()
            temp0 = df.ix[df['income'] == '>50K', col].value_counts() 
            
            ylim = [0, max(max(temp1), max(temp0))]
            fig = plt.figure(figsize = (12,6))
            fig.clf()
            ax1 = fig.add_subplot(1, 2, 1)
            ax0 = fig.add_subplot(1, 2, 2) 
            temp1.plot(kind = 'bar', ax = ax1, ylim = ylim)
            ax1.set_title('Values of ' + col + '\n for income <= 50K')
            temp0.plot(kind = 'bar', ax = ax0, ylim = ylim)
            ax0.set_title('Values of ' + col + '\n for income > 50K')
    return('Done')        

income_barplot(Income)

Examine the plots.

Note that marital.status may have some power in predicting the label category. The proportions of cases are quite different for people with income <=50K and people with income >50K.  Husband dominates >50K, while Not-in-family is the largest fraction for <=50K. This feature is therefore said to separate the two label categories. However, the separation is not perfect as there is overlap in the cases. This is typical and it is rare to find a single feature which provides perfect separation.

Conversely, native.country may not be that useful in predicting the label category. First, there are quite a few categories. Second, the only category with significant numbers of people is United States. There is only a small fraction of the people spread over the other 41 countries. There is no particular pattern noticeable between people with income <=50K and people with income >50K. 

Review the plots for other features. Consider if these features help to separate the label categories.
 
Note: When looking at the bar plots of the features conditioned on label value, keep in mind that there are about three times as many people with low income as high income. Therefore, it is the proportions between the categories of the label which are important, not the absolute values. In other words, are the particular values of a feature relatively more or less likely for the label categories? 

### Box Plot the Numeric Features
There are two possible categories or label values for a personâs income category, in this case, <=50K or >50K. In this exercise you will create box plots for each numeric feature, conditioned on the label value. By examining these box plots, you will determine which features are likely to be useful as predictors of the label categories. 

Run the code below to define a function that creates box plots of numeric features vs income, and then call the function with income features.

In [ ]:
## Plot categorical variables as box plots
def income_boxplot(df):
    import numpy as np
    import matplotlib.pyplot as plt
    
    cols = df.columns.tolist()[:-1]
    for col in cols:
        if(df[col].dtype in [np.int64, np.int32, np.float64]):                  
            fig = plt.figure(figsize = (6,6))
            fig.clf()
            ax = fig.gca() 
            df.boxplot(column = [col], ax = ax, by = ['income'])          
    return('Done') 

income_boxplot(Income)

Reviwe the plots.

Note that education.num may have some power in predicting the label category. People with income >50K tend to have consistently higher education.num values than people with income <=50K.

Conversely, capital.loss is not likely to be that useful in predicting the label category. Almost all people in the sample have zero (0) capital loss for both label values. Note that the quartile markers in the box plot all fall on top of each other at 0.

Review the plots for other features and consider if these features help to separate the label categories.
